In [101]:
import pandas as pd
import numpy as np
import pandas_profiling
from datetime import date as dt

In [102]:
seg_df = pd.read_excel('Customers.xlsx')

In [103]:
seg_df.head()

,Customer_ID,Gender,First_Date,First_region,First_state,First_city,Recent_Date,Last_region,Last_state,Last_city,Customer_Value,Buy_Times,Points_Earned,Points_Redeemed
0,129609144,M,20130130,West,Gujarat,Surat,20130216,West,Gujarat,Surat,2032.0,2,200.0,0.0
1,129627580,M,20130129,North,Delhi,Delhi,20130129,North,Delhi,Delhi,1709.0,1,100.0,0.0
2,130363481,M,20130221,West,Gujarat,Surat,20130221,West,Gujarat,Surat,523.0,1,10.0,0.0
3,49817480,M,20130222,West,Maharashtra,Pimpri-Chinchwad,20130222,West,Maharashtra,Pimpri-Chinchwad,5132.0,1,170.0,0.0
4,126343829,F,20130301,North,Delhi,Delhi,20130301,North,Delhi,Delhi,1008.0,1,60.0,0.0


In [15]:
report=pandas_profiling.ProfileReport(seg_df)
report.to_file('Profile_report.html')

In [104]:
seg_df['Recent_Date'] = pd.to_datetime(seg_df['Recent_Date'].astype(str), format='%Y%m%d',infer_datetime_format=True)
seg_df['First_Date'] = pd.to_datetime(seg_df['First_Date'].astype(str), format='%Y%m%d',infer_datetime_format=True)
seg_df['Current_Date']=pd.to_datetime("now",format='%Y%m%d',infer_datetime_format=True)
seg_df['recentdays']=seg_df['Current_Date']-seg_df['Recent_Date']

In [105]:
seg_df.dtypes

Customer_ID                  int64
Gender                      object
First_Date          datetime64[ns]
First_region                object
First_state                 object
First_city                  object
Recent_Date         datetime64[ns]
Last_region                 object
Last_state                  object
Last_city                   object
Customer_Value             float64
Buy_Times                    int64
Points_Earned              float64
Points_Redeemed            float64
Current_Date        datetime64[ns]
recentdays         timedelta64[ns]
dtype: object

In [106]:
seg_df.columns

Index(['Customer_ID', 'Gender', 'First_Date', 'First_region', 'First_state',
       'First_city', 'Recent_Date', 'Last_region', 'Last_state', 'Last_city',
       'Customer_Value', 'Buy_Times', 'Points_Earned', 'Points_Redeemed',
       'Current_Date', 'recentdays'],
      dtype='object')

In [107]:
seg_df.rename(columns={'recentdays': 'recency', 
                         'Buy_Times': 'frequency', 
                         'Customer_Value': 'monetary'}, inplace=True)

In [108]:
# creating Deciles
seg_df['decile'] = pd.qcut(seg_df['monetary'], 10, labels=False)

In [109]:
seg_df['recency']=seg_df.recency.dt.days
seg_df.head()

,Customer_ID,Gender,First_Date,First_region,First_state,First_city,Recent_Date,Last_region,Last_state,Last_city,monetary,frequency,Points_Earned,Points_Redeemed,Current_Date,recency,decile
0,129609144,M,2013-01-30,West,Gujarat,Surat,2013-02-16,West,Gujarat,Surat,2032.0,2,200.0,0.0,2019-10-12 19:31:44.501658,2429,3.0
1,129627580,M,2013-01-29,North,Delhi,Delhi,2013-01-29,North,Delhi,Delhi,1709.0,1,100.0,0.0,2019-10-12 19:31:44.501658,2447,2.0
2,130363481,M,2013-02-21,West,Gujarat,Surat,2013-02-21,West,Gujarat,Surat,523.0,1,10.0,0.0,2019-10-12 19:31:44.501658,2424,0.0
3,49817480,M,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,5132.0,1,170.0,0.0,2019-10-12 19:31:44.501658,2423,5.0
4,126343829,F,2013-03-01,North,Delhi,Delhi,2013-03-01,North,Delhi,Delhi,1008.0,1,60.0,0.0,2019-10-12 19:31:44.501658,2416,1.0


In [110]:
seg_df.decile.value_counts()

8.0    997
5.0    997
0.0    997
3.0    997
9.0    996
4.0    996
7.0    996
6.0    996
1.0    996
2.0    996
Name: decile, dtype: int64

In [111]:
col_kpi=['monetary','recency','frequency','Points_Earned','Points_Redeemed']

In [112]:
VB_Profiling=seg_df.groupby(['decile']).apply(lambda x: x[col_kpi].mean())

In [113]:
profiling = VB_Profiling.T
profiling

decile,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
monetary,682.684037,1064.327316,1502.205026,2293.104905,3315.710046,4765.670628,7126.828173,11314.674592,19910.764392,73495.590095
recency,2622.166499,2589.941767,2575.538153,2554.177533,2562.486948,2512.625878,2485.770080,2463.354418,2449.820461,2437.355422
frequency,1.040120,1.017068,1.184739,1.740221,2.468876,2.964895,4.297189,6.521084,10.311936,23.226908
Points_Earned,44.289447,77.875502,96.586519,141.322969,189.236948,294.142570,401.805051,630.693467,1006.773414,2416.758829
Points_Redeemed,60.857430,25.177711,27.233936,30.646586,42.159639,53.332999,62.135815,153.079397,344.024072,5044.182093


In [114]:
profiling.to_csv('profiling.csv')

In [115]:
#Method-2: Creating quartiles

quantiles = seg_df[['frequency','monetary','recency' ]].quantile(q=[0.25,0.5,0.75])
quantiles

,frequency,monetary,recency
0.25,1.0,1484.5000,2413.0
0.50,2.0,3960.0005,2482.0
0.75,6.0,11136.0000,2614.0


In [116]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 6.0},
 'monetary': {0.25: 1484.5, 0.5: 3960.0005, 0.75: 11136.0},
 'recency': {0.25: 2413.0, 0.5: 2482.0, 0.75: 2614.0}}

In [117]:
#We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good.
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [118]:
seg_df['R_Quartile'] = seg_df['recency'].apply(RClass, args=('recency',quantiles,))
seg_df['F_Quartile'] = seg_df['frequency'].apply(FMClass, args=('frequency',quantiles,))
seg_df['M_Quartile'] = seg_df['monetary'].apply(FMClass, args=('monetary',quantiles,))

In [119]:
seg_df.head()

,Customer_ID,Gender,First_Date,First_region,First_state,First_city,Recent_Date,Last_region,Last_state,Last_city,monetary,frequency,Points_Earned,Points_Redeemed,Current_Date,recency,decile,R_Quartile,F_Quartile,M_Quartile
0,129609144,M,2013-01-30,West,Gujarat,Surat,2013-02-16,West,Gujarat,Surat,2032.0,2,200.0,0.0,2019-10-12 19:31:44.501658,2429,3.0,2,3,3
1,129627580,M,2013-01-29,North,Delhi,Delhi,2013-01-29,North,Delhi,Delhi,1709.0,1,100.0,0.0,2019-10-12 19:31:44.501658,2447,2.0,2,4,3
2,130363481,M,2013-02-21,West,Gujarat,Surat,2013-02-21,West,Gujarat,Surat,523.0,1,10.0,0.0,2019-10-12 19:31:44.501658,2424,0.0,2,4,4
3,49817480,M,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,5132.0,1,170.0,0.0,2019-10-12 19:31:44.501658,2423,5.0,2,4,2
4,126343829,F,2013-03-01,North,Delhi,Delhi,2013-03-01,North,Delhi,Delhi,1008.0,1,60.0,0.0,2019-10-12 19:31:44.501658,2416,1.0,2,4,4


In [120]:
seg_df1=seg_df.assign(R_seg=np.nan, FM_seg =np.nan)
seg_df1.R_seg[seg_df['R_Quartile']==4] = "3.Churn"
seg_df1.R_seg[seg_df['R_Quartile']==2] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==3] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==1] = "1.Active"

seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==1)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==2)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==3)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==3)] = "0.Standard"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==4)] = "0.Standard"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==4)] = "0.Standard"

seg_df1.FM_seg.fillna("4.Premium", inplace=True)

In [121]:
seg_df1.head(5)

,Customer_ID,Gender,First_Date,First_region,First_state,First_city,Recent_Date,Last_region,Last_state,Last_city,...,Points_Earned,Points_Redeemed,Current_Date,recency,decile,R_Quartile,F_Quartile,M_Quartile,R_seg,FM_seg
0,129609144,M,2013-01-30,West,Gujarat,Surat,2013-02-16,West,Gujarat,Surat,...,200.0,0.0,2019-10-12 19:31:44.501658,2429,3.0,2,3,3,2.At Risk,1.Silver
1,129627580,M,2013-01-29,North,Delhi,Delhi,2013-01-29,North,Delhi,Delhi,...,100.0,0.0,2019-10-12 19:31:44.501658,2447,2.0,2,4,3,2.At Risk,0.Standard
2,130363481,M,2013-02-21,West,Gujarat,Surat,2013-02-21,West,Gujarat,Surat,...,10.0,0.0,2019-10-12 19:31:44.501658,2424,0.0,2,4,4,2.At Risk,0.Standard
3,49817480,M,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,2013-02-22,West,Maharashtra,Pimpri-Chinchwad,...,170.0,0.0,2019-10-12 19:31:44.501658,2423,5.0,2,4,2,2.At Risk,1.Silver
4,126343829,F,2013-03-01,North,Delhi,Delhi,2013-03-01,North,Delhi,Delhi,...,60.0,0.0,2019-10-12 19:31:44.501658,2416,1.0,2,4,4,2.At Risk,0.Standard


In [122]:
# Key performace variable selection
col_kpi=['monetary','recency','frequency','Points_Earned','Points_Redeemed']

In [123]:
#Profiling
Profiling=seg_df1.groupby(['R_seg', 'FM_seg']).apply(lambda x: x[col_kpi].mean()).T

In [124]:
Profiling

R_seg               1.Active                                          \
FM_seg            0.Standard     1.Silver       2.Gold     4.Premium   
monetary         1323.900607  6415.575523  9327.173999  42799.504176   
recency          2398.225610  2398.021277  2398.237069   2396.791266   
frequency           1.021341     3.933511     4.310345     20.315284   
Points_Earned      81.452599   317.164894   479.059078   1810.092105   
Points_Redeemed     0.000000   119.449468    44.313218   1688.043745   

R_seg              2.At Risk                                           \
FM_seg            0.Standard     1.Silver        2.Gold     4.Premium   
monetary         1367.005175  5442.911248  10459.379023  33599.386574   
recency          2508.143282  2494.887511   2485.518072   2461.201065   
frequency           1.024376     2.686448      4.043940     14.468708   
Points_Earned      85.456599   284.694765    444.342330   1337.468541   
Points_Redeemed     5.626040    48.675532     69.417614   3863.920000   

R_seg                3.Churn                                            
FM_seg            0.Standard     1.Silver        2.Gold      4.Premium  
monetary         1249.043969  6813.650655  10311.277825  158196.761288  
recency          2737.352691  2712.916115   2709.063717    2671.961538  
frequency           1.021246     1.997792      3.706195      17.923077  
Points_Earned      88.518440   277.747788    367.234043    3726.826923  
Points_Redeemed    75.491141    51.915743    113.040853   10698.307692

In [125]:
Profiling.to_csv('Profiling_rfm.csv')